# Line Color mapper

Bokeh has `mapper` that allow to change color with data if they support it.

It is possible to do with it Heatmap, where you put `fill_color={"field": "YourValueToMap", "mapper":ColorMapperObject}`.

In the case of a line, it is not possible, as the attribute must be a `Color` (not a `ColorSpec`)

However, `bokeh.plotting.segment` accept this class.

The trick will allow to change color ! 

![Sinusoidal](color_mapper_bokeh_plot.png)

In [1]:
from bokeh.io import show
from bokeh.palettes import Viridis256
from bokeh.plotting import figure
from bokeh.models   import ColumnDataSource

In [2]:
import numpy as np
import pandas as pd

In [3]:
x  = np.linspace(0, 2 * np.pi * 4, 100)
y1 = np.cos(x)
y2 = np.sin(x)

In [4]:
def LinearColorMapper(serie, palette, range=None):
    r1, r2 = (0, 0)
    if range is not None:
        r1, r2 = range
    else:
        possible = np.where(~np.isnan(serie))[0]
        r1, r2 = min(serie), max(serie)
    
    l = len(palette) -1
    i_serie = np.array(serie)
    i_serie[i_serie > r2] = r2
    i_serie[i_serie < r1] = r1
    
    i_serie = ((serie - r1)/(r2-r1)*l).astype(int)
    return list(map(lambda x: palette[x], i_serie))
        

In [5]:
source = ColumnDataSource(data={"x0": x[:-1], 
                                "x1": x[1:],
                                "f0": y1[:-1],
                                "f1": y1[1:],
                                "af0": y2[:-1],
                                "af1": y2[1:],
                                "col": LinearColorMapper(y2[1:], Viridis256)                               
                               })

In [6]:
p = figure(title="My Linearly mapped color",
          width=800)
p.segment(x0="x0", x1="x1", y0="f0",y1="f1", 
          source=source, 
          line_color={"field":"col"},
          line_width=3)

p.segment(x0="x0", x1="x1", y0="af0",y1="af1", 
          source=source, 
          line_color={"field":"col"},
         )

show(p)